In [12]:
!pip install matplotlib seaborn openpyxl

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Veriyi Yükle
df = pd.read_excel("arcelik_master_temiz_analiz.xlsx")

# 2. STRATEJİK KELİME GRUPLARI (Daha hassas ayrım)
keywords = {
    # ÜRETİM/TEKNİK HATALAR (Kök Neden: Fabrika)
    'Fabrika_Teknik': ['motor', 'kart', 'gaz', 'sızıntı', 'defo', 'pas', 'arıza', 'bozuk', 'kronik', 
                       'paslanma', 'boya', 'ekran', 'sensör', 'rezistans', 'pompa', 'tambur', 'kazan', 
                       'kompresör', 'çatlak', 'kırık', 'ayıplı', 'kusurlu', 'ses', 'gürültü'],
    
    # DAVRANIŞSAL/SÜREÇSEL HATALAR (Kök Neden: Personel/Hizmet)
    'Hizmet_Hata': ['davranış', 'ilgisiz', 'kaba', 'yalan', 'tavır', 'üslup', 'saygısız', 'muhatap', 
                    'bilgisiz', 'gelmedi', 'bekletti', 'yanlış bilgi', 'yüzüme kapattı', 'çözemedi'],

    # DEPARTMAN BELİRLEYİCİLER (Darboğaz Analizi İçin)
    'Dep_Servis': ['servis', 'tamir', 'onarım', 'montaj', 'kurulum', 'bakım', 'teknisyen', 'usta'],
    'Dep_Cagri': ['telefon', 'çağrı', 'müşteri hizmetleri', 'temsilci', 'bağlanamadım', 'aradım', 'mh'],
    'Dep_Lojistik': ['nakliye', 'kargo', 'teslim', 'lojistik', 'taşıma', 'sevkiyat', 'horoz', 'borusan'],
    'Dep_Satis': ['bayi', 'mağaza', 'şube', 'satış', 'satıcı', 'teşhir', 'gizerler', 'istinye'],
    'Dep_Finans': ['fatura', 'taksit', 'ödeme', 'iade ücreti', 'para iadesi', 'bedel iadesi'],

    # KRİZ/CHURN GÖSTERGELERİ
    'Kritik_Churn': ['iade', 'thh', 'hakem heyeti', 'avukat', 'dava', 'pişman', 'asla', 'tüketici', 'yasal', 'mahkeme']
}

def balanced_classify(row):
    text = str(row['Temiz_Metin']).lower()
    scores = {key: sum(1 for word in words if word in text) for key, words in keywords.items()}
    
    # Kök Neden Analizi: Sadece Teknik vs Hata kelimelerini karşılaştırır
    if scores['Fabrika_Teknik'] > scores['Hizmet_Hata']:
        kok = 'Fabrika (Üretim)'
    elif scores['Hizmet_Hata'] > scores['Fabrika_Teknik']:
        kok = 'Personel (Hizmet)'
    elif scores['Fabrika_Teknik'] > 0: # Eşitlik durumunda teknik baskınsa
        kok = 'Fabrika (Üretim)'
    else:
        kok = 'Belirsiz/Karma'
    
    # Darboğaz Analizi: Hangi departman daha çok anılıyor?
    dept_scores = {
        'Teknik Servis': scores['Dep_Servis'],
        'Çağrı Merkezi': scores['Dep_Cagri'],
        'Lojistik': scores['Dep_Lojistik'],
        'Satış & Bayi': scores['Dep_Satis'],
        'Muhasebe & Finans': scores['Dep_Finans']
    }
    max_dept = max(dept_scores, key=dept_scores.get)
    darbogaz = max_dept if dept_scores[max_dept] > 0 else 'Belirsiz'
    
    return pd.Series([kok, darbogaz, 'Yüksek' if scores['Kritik_Churn'] > 0 else 'Düşük'])

# Uygulama
df[['Kok_Neden', 'Darbogaz_Departman', 'Churn_Riski']] = df.apply(balanced_classify, axis=1)
print("✅ Sınıflandırma dengelendi.")

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Grafik Stil Ayarları
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

# --- GRAFİK 1: Kök Neden (Pasta) ---
plt.figure(figsize=(7,7))
kok_counts = df['Kok_Neden'].value_counts()
plt.pie(kok_counts, labels=kok_counts.index, autopct='%1.1f%%', colors=['#ff9999', '#66b3ff', '#99ff99'], startangle=140)
plt.title('Şikayetlerin Kök Neden Dağılımı', fontsize=14, fontweight='bold')
plt.show()

# --- GRAFİK 2: Departman Darboğazı (Bar) ---
plt.figure(figsize=(10,6))
dept_data = df[df['Darbogaz_Departman'] != 'Belirsiz']['Darbogaz_Departman'].value_counts(normalize=True) * 100
sns.barplot(x=dept_data.index, y=dept_data.values, palette='viridis')
plt.title('Departman Bazlı Operasyonel Darboğazlar (%)', fontsize=14, fontweight='bold')
plt.ylabel('Yüzde (%)')
plt.xticks(rotation=45)
plt.show()

# --- GRAFİK 3: Kategoriye Göre Churn Riski ---
plt.figure(figsize=(12,6))
sns.countplot(data=df, x='Kategori', hue='Churn_Riski', palette='coolwarm')
plt.title('Ürün Kategorilerine Göre Müşteri Kaybı (Churn) Riski', fontsize=14, fontweight='bold')
plt.legend(title='Risk Seviyesi')
plt.show()

# İstatistik Özeti
print("\n--- ANALİZ SONUÇLARI ---")
print(f"Fabrika Hatası Oranı: %{df['Kok_Neden'].value_counts(normalize=True).get('Fabrika (Üretim)', 0)*100:.1f}")
print(f"En Büyük Darboğaz: {dept_data.idxmax()} (%{dept_data.max():.1f})")

ModuleNotFoundError: No module named 'matplotlib'